## Partie 1 - Description du problème

En ce qui concernce le football, le pari sportif est basé sur l'ignorance du resultat d'un match opposant deux equipe. <br> Reussir a predire qu'un match se termine par une victoire ou une defaite d'une equipe ou un nul des deux serait le gral du parie.<br> Dans de ce notebook, nous allons creer un classifieur qui nous permettra de predire les resultats de matchs avec un certains taux de succes, et ceux en nous basant sur les données collectés sur des equipes de 11 leagues europeenes et sur plusieurs saisons.

## Preparation des tables

In [11]:
import pandas as pd


In [12]:
country = pd.read_csv("soccer/country.csv")
league = pd.read_csv("soccer/league.csv")
player = pd.read_csv("soccer/player.csv")
team = pd.read_csv("soccer/team.csv")
playerattributes = pd.read_csv("soccer/playerattributes.csv")
teamattributes = pd.read_csv("soccer/teamattributes.csv")
match = pd.read_csv("soccer/match.csv")


## Partie 2 - Modéle

** importation des modules necessaires **

In [13]:
import random
import numpy as np
import pandas as pd
import matplotlib as plt
from math import *
#import graphviz
import datetime

### arbre de decision

* **Implementation du LabeledSet**

In [15]:
class LabeledSet:  
    
    def __init__(self, input_dimension):
        self.input_dimension = input_dimension
        self.nb_examples = 0
    
    def addExample(self,vector,label):
        if (self.nb_examples == 0):
            self.x = np.array([vector])
            self.y = np.array([label])
        else:
            self.x = np.vstack((self.x, vector))
            self.y = np.vstack((self.y, label))
        
        self.nb_examples = self.nb_examples + 1
    
    #Renvoie la dimension de l'espace d'entrée
    def getInputDimension(self):
        return self.input_dimension
    
    #Renvoie le nombre d'exemples dans le set
    def size(self):
        return self.nb_examples
    
    #Renvoie la valeur de x_i
    def getX(self, i):
        return self.x[i]
        
    
    #Renvouie la valeur de y_i
    def getY(self, i):
        return(self.y[i])

* **Implementation des methodes pour l'arbre de decision**

In [5]:
def classe_majoritaire(labeledSet):
    pos,neg,zero=0,0,0
    for i in range(labeledSet.size()):
        if(labeledSet.getY(i)==1):
            pos+=1
        elif(labeledSet.getY(i)==-1):
            neg+=1
        else:
            zero+=1
    l=[neg,zero,pos]
    return (np.argmax(l)-1)

In [6]:
def shannon(listeProb):
    entropie = 0
    for p in listeProb:
        if p == 0 :
            continue
        k = len(listeProb)
        loga = log(p, k)
        entropie += (p*loga)
    return (-1)*entropie

In [7]:
def entropie(labeledSet):
    pos,neg,zero=0,0,0
    for i in range(labeledSet.size()):
        if(labeledSet.getY(i)==1):
            pos+=1
        elif(labeledSet.getY(i)==-1):
            neg+=1
        else:
            zero+=1
    res = list()
    res.append(pos/labeledSet.size())
    res.append(zero/labeledSet.size())
    res.append(neg/labeledSet.size())
    return shannon(res)

In [8]:
def discretise(LSet, col):
    """ LabelledSet * int -> tuple[float, float]
        col est le numÃ©ro de colonne sur X Ã  discrÃ©tiser
        rend la valeur de coupure qui minimise l'entropie ainsi que son entropie.
    """
    # initialisation:
    min_entropie = 1.1  # on met Ã  une valeur max car on veut minimiser
    min_seuil = 0.0     
    # trie des valeurs:
    ind= np.argsort(LSet.x,axis=0)
    
    # calcul des distributions des classes pour E1 et E2:
    inf_plus  = 0               # nombre de +1 dans E1
    inf_moins = 0               # nombre de -1 dans E1
    inf_zeros=0                 # nombre de 0 dans E1
    sup_plus  = 0               # nombre de +1 dans E2
    sup_moins = 0               # nombre de -1 dans E2
    sup_zeros=0                 # nombre de 0 dans E2

    # remarque: au dÃ©part on considÃ¨re que E1 est vide et donc E2 correspond Ã  E. 
    # Ainsi inf_plus et inf_moins et inf_zeros valent 0. Il reste Ã  calculer sup_plus,sup_moins et sup_zeros 
    # dans E.
    for j in range(0,LSet.size()):
        if (LSet.getY(j) == -1):
            sup_moins += 1
        elif(LSet.getY(j) == +1):
            sup_plus += 1
        else:
            sup_zeros += 1
            
    nb_total = (sup_plus + sup_moins + sup_zeros) # nombre d'exemples total dans E
    
    # parcours pour trouver le meilleur seuil:
    for i in range(len(LSet.x)-1):
        v_ind_i = ind[i]   #Â vecteur d'indices
        courant = LSet.getX(v_ind_i[col])[col]
        lookahead = LSet.getX(ind[i+1][col])[col]
        val_seuil = (courant + lookahead) / 2.0;
        # M-A-J de la distrib. des classes:
        # pour rÃ©duire les traitements: on retire un exemple de E2 et on le place
        # dans E1, c'est ainsi que l'on dÃ©place donc le seuil de coupure.
        if LSet.getY(ind[i][col])[0] == -1:
            inf_moins += 1
            sup_moins -= 1
        elif(LSet.getY(ind[i][col])[0] == +1):
            inf_plus += 1
            sup_plus -= 1
        else:
            inf_zeros += 1
            sup_zeros -=1
            
            
        # calcul de la distribution des classes de chaque cÃ´tÃ© du seuil:
        nb_inf = (inf_moins + inf_plus + inf_zeros)*1.0     #Â rem: on en fait un float pour Ã©viter
        nb_sup = (sup_moins + sup_plus + sup_zeros)*1.0     # que ce soit une division entiÃ¨re.
        # calcul de l'entropie de la coupure
        val_entropie_inf = shannon([inf_moins / nb_inf, inf_plus  / nb_inf, inf_zeros / nb_inf])
        val_entropie_sup = shannon([sup_moins / nb_sup, sup_plus  / nb_sup, sup_zeros / nb_sup])
        val_entropie = (nb_inf / nb_total) * val_entropie_inf + (nb_sup / nb_total) * val_entropie_sup
        # si cette coupure minimise l'entropie, on mÃ©morise ce seuil et son entropie:
        if (min_entropie > val_entropie):
            min_entropie = val_entropie
            min_seuil = val_seuil
    return (min_seuil, min_entropie)

In [9]:
def divise(LSet,att,seuil):
    ind = np.argsort(LSet.x,axis=0)
    x = LSet.x[ind[0:len(LSet.x),att]]
    E1,E2=LabeledSet(2),LabeledSet(2)
    for i in range(len(x)):
        indice_courant = ind[i][att]
        if x[i][att] < seuil:
            E1.addExample(LSet.getX(indice_courant), LSet.getY(indice_courant))
        else :
            E2.addExample(LSet.getX(indice_courant), LSet.getY(indice_courant))
    return E1,E2

* **Implementation d'un classifieur**

In [10]:
class Classifier:
    def __init__(self,input_dimension):
        """ Constructeur """
        raise NotImplementedError("Please Implement this method")
    
    
    # Permet de calculer la prediction sur x => renvoie un score
    def predict(self,x):
        raise NotImplementedError("Please Implement this method")

    
    # Permet d'entrainer le modele sur un ensemble de donnÃ©es
    def train(self,labeledSet):
        raise NotImplementedError("Please Implement this method")
    
    #Permet de calculer la qualitÃ© du systÃ¨me 
    def accuracy(self, dataset):
        nb_ok = 0
        for i in range(dataset.size()):
            output = self.predict(dataset.getX(i))
            if (output * dataset.getY(i) > 0):
                nb_ok = nb_ok + 1
        acc = nb_ok / (dataset.size() * 1.0)
        return acc    

* **Implementation du Modele d'arbre de décision **

In [11]:
class ArbreDecision(Classifier):
    # Constructeur
    def __init__(self,epsilon):
        #Â valeur seuil d'entropie pour arrÃªter la construction
        self.epsilon= epsilon
        self.racine = None
    
    # Permet de calculer la prediction sur x => renvoie un score
    def predict(self,x):
        # classification de l'exemple x avec l'arbre de dÃ©cision
        # on rend 0 (classe -1) ou 1 (classe 1)
        classe = self.racine.classifie(x)
        if (classe == 1):
            return(1)
        elif(classe == -1):
            return(-1)
        else:
            return 0 
    # Permet d'entrainer le modele sur un ensemble de donnÃ©es
    def train(self,set):
        # construction de l'arbre de dÃ©cision 
        self.set=set
        self.racine = construit_AD(set,self.epsilon)
    
    def validation(self,dataSet):
        valid = 0
        for i in range(dataSet.size()):
            output = self.predict(dataSet.getX(i))
            if (output * dataSet.getY(i) > 0):
                valid = valid + 1
        acc = valid / (dataSet.size() * 1.0)
        return acc

    # Permet d'afficher l'arbre
    def plot(self):
        gtree = gv.Digraph(format='png')
        return self.racine.to_graph(gtree)
       

In [12]:

class ArbreBinaire:
    def __init__(self):
        self.attribut = None   # numÃ©ro de l'attribut
        self.seuil = None
        self.inferieur = None # ArbreBinaire Gauche (valeurs <= au seuil)
        self.superieur = None # ArbreBinaire Gauche (valeurs > au seuil)
        self.classe = None # Classe si c'est une feuille: -1 ou +1
        
    def est_feuille(self):
        """ rend True si l'arbre est une feuille """
        return self.seuil == None
    
    def ajoute_fils(self,ABinf,ABsup,att,seuil):
        """ ABinf, ABsup: 2 arbres binaires
            att: numÃ©ro d'attribut
            seuil: valeur de seuil
        """
        self.attribut = att
        self.seuil = seuil
        self.inferieur = ABinf
        self.superieur = ABsup
    
    def ajoute_feuille(self,classe):
        """ classe: -1 ou + 1
        """
        self.classe = classe
        
    def classifie(self,exemple):
        """ exemple : numpy.array
            rend la classe de l'exemple: +1 ou -1
        """
        if self.est_feuille():
            return self.classe
        if exemple[self.attribut] <= self.seuil:
            return self.inferieur.classifie(exemple)
        return self.superieur.classifie(exemple)
    
    def to_graph(self, g, prefixe='A'):
        """ construit une reprÃ©sentation de l'arbre pour pouvoir
            l'afficher
        """
        if self.est_feuille():
            g.node(prefixe,str(self.classe),shape='box')
        else:
            g.node(prefixe, str(self.attribut))
            self.inferieur.to_graph(g,prefixe+"g")
            self.superieur.to_graph(g,prefixe+"d")
            g.edge(prefixe,prefixe+"g", '<='+ str(self.seuil))
            g.edge(prefixe,prefixe+"d", '>'+ str(self.seuil))
        
        return g


In [13]:
def construit_AD(LSet,epsilon):
    
    ab = ArbreBinaire()
    if LSet.size() == 0:
        ab.ajoute_feuille(0)
    elif entropie(LSet) <= epsilon:
        ab.ajoute_feuille(classe_majoritaire(LSet))
    else:
        seuilX1, entropieX1 = discretise(LSet,0)
        seuilX2, entropieX2 = discretise(LSet,1)
        if entropieX1 < entropieX2:
            E1, E2 = divise(LSet,0,seuilX1)
            ABinf = construit_AD(E1,epsilon)
            ABsup = construit_AD(E2,epsilon)
            ab.ajoute_fils(ABinf,ABsup,0,seuilX1)
        else:
            E1, E2 = divise(LSet,1,seuilX2)
            ABinf = construit_AD(E1,epsilon)
            ABsup = construit_AD(E2,epsilon)
            ab.ajoute_fils(ABinf,ABsup,1,seuilX2)
    return ab

## Partie 3  - Code

** Méthodes de traitement des données **

* Methode de creation du labeledSet sous forme (X,Y) avec: X=>(home_team_api_d,away_team_api_id), Y=> resultat pour l'equipe a domicile (+1 victoire,0nul,-1Défaite)

In [18]:
def matchsaison(saison,numleague):
    bdtot= LabeledSet(2)
    for i in range(len(match)):
        
        
        if(match.iloc[i]["league_id"]==numleague and saison==str(match.iloc[i]["season"])):
            res=match.iloc[i]["home_team_goal"]-match.iloc[i]["away_team_goal"]
            lab=0
            if (res>0):
                lab=+1
            elif(res<0):
                lab=-1
            else:
                lab=0
            bdtot.addExample((match.iloc[i]['home_team_api_id'],match.iloc[i]['away_team_api_id']),lab)
            #print(match.iloc[i]['home_team_api_id'],match.iloc[i]['away_team_api_id'],lab)
    return bdtot

* Récuperer les matchs joué dans un ligue pendant une saison particuliere

In [19]:
def MatchesSaisonLeague(championnat,saison):
    
   
    #avoir les matches joué a domicile pendant la saison en question
    gb=match.groupby("league_id")
    matchesSaisonLeague=gb.get_group(championnat)
    matchesSaisonLeague=matchesSaisonLeague[matchesSaisonLeague.season == saison]
    matchesSaisonLeague=matchesSaisonLeague[["home_team_api_id","away_team_api_id","date",'home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
               "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
               "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
               "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]]
    
    return matchesSaisonLeague

* Recuperer les id et les notes des joueurs pendant une saison

In [20]:
def StatJoueurSaison(saison):
    
    playerattributes['date']  = pd.to_datetime(playerattributes['date'])
    df_filtre = playerattributes[(playerattributes.date > datetime.date(int(saison[0:4]),9,1)) & (playerattributes.date < datetime.date(int(saison[5:]),8,30))]
    df_filtre=df_filtre[['player_api_id','overall_rating']]
    
    return df_filtre

* Calculer les notes des equipes jouant dans une league pendant une saison particuliere

In [21]:
def StatEquipesLeagueSaison(saison,championnat):
    matchesSaisonLeague=MatchesSaisonLeague(championnat,saison)
    statJoueur=StatJoueurSaison(saison)
    statEquipes=pd.DataFrame(({'team_api_id': [], 'overall_rating': []}))
    ColonnesJoueurs_home = ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
                           "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
                           "home_player_11"]


    #Pour recuperer les joueurs qui font partie des 11 titulaires des matches a l'exterieur
    ColonnesJoueurs_away=["away_player_1", "away_player_2", "away_player_3", "away_player_4","away_player_5",
                          "away_player_6", "away_player_7","away_player_8", "away_player_9","away_player_10",
                          "away_player_11"]


    #avoir les matches joué a domicile pendant la saison en question
    
   


    #avoir les matches joué a l'exterieur pendant la saison en question
    
    
    ligne=0
    
    for matche in matchesSaisonLeague.itertuples():
        #print(type(matche))
        #sam = statEquipes.loc[statEquipes["team_api_id"] == matche["home_team_api_id"]]
        #lig, col =sam.shape
        #print(lig)
        if matche.home_team_api_id in list(statEquipes.team_api_id):
            continue
        else:
            
            gb=matchesSaisonLeague.groupby("home_team_api_id")
            myteam_home=gb.get_group(matche.home_team_api_id)
            
           
            gb2=matchesSaisonLeague.groupby("away_team_api_id")
            myteam_away=gb.get_group(matche.home_team_api_id)
            NotesJoueursD=pd.DataFrame()
            for postejoueur in ColonnesJoueurs_home:
                # avoir la liste des joueurs ayant joué dans ce poste

                effectifs=myteam_home[postejoueur].dropna() # on utilise dropna pour eliminer tous les nan
                #print(effectifs)
                NotesJoueursD[postejoueur]=[0]
                noteglob=pd.DataFrame()
                note=list()
                for i in range(len(effectifs)):
                    noteglob=pd.DataFrame()

                    df_filtre=statJoueur[['player_api_id','overall_rating']]
                    note.append(df_filtre[df_filtre['player_api_id']==effectifs.iloc[i]]['overall_rating'].mean())              
                    #print(df_masked[df_masked['player_api_id']==effectifs.iloc[i]])
                    #NotesJoueurs[postejoueur]=df_masked[df_masked['player_api_id']==effectifs.iloc[i]]['overall_rating']

                noteglob = pd.DataFrame(np.array(note), columns = ["a"]).dropna()

                NotesJoueursD[postejoueur]=[noteglob.mean()[0]]
            #print(NotesJoueursD)

            NotesJoueursA=pd.DataFrame()
            for postejoueur in ColonnesJoueurs_away:
                # avoir la liste des joueurs ayant joué dans ce poste

                effectifs=myteam_away[postejoueur].dropna() # on utilise dropna pour eliminer tous les nan

                NotesJoueursA[postejoueur]=[0]
                noteglob=pd.DataFrame()
                note=list()
                for i in range(len(effectifs)):
                    noteglob=pd.DataFrame()

                    note.append(df_filtre[df_filtre['player_api_id']==effectifs.iloc[i]]['overall_rating'].mean())              
                    #print(df_masked[df_masked['player_api_id']==effectifs.iloc[i]])
                    #NotesJoueurs[postejoueur]=df_masked[df_masked['player_api_id']==effectifs.iloc[i]]['overall_rating']
                noteglob = pd.DataFrame(np.array(note), columns = ["a"]).dropna()

                NotesJoueursA[postejoueur]=[noteglob.mean()[0]]
            #print(NotesJoueursA.mean(axis = 1))
        statEquipes.loc[ligne]=[matche.home_team_api_id, ((NotesJoueursA.mean(axis = 1)+NotesJoueursD.mean(axis = 1))/2)[0]]
        ligne+=1
    
    
    statEquipes.rename(columns={'overall_rating': 'team_api_id', 'team_api_id': 'overall_rating'}, inplace=True)
   
    return statEquipes.groupby('team_api_id', as_index=False).max()
                        
    
    

* Calcule de la note d'une equipe pendant une saison

In [22]:
def noteEquipe(idequipe,saison):
    #Pour recuperer les joueurs qui font partie des 11 titulaires des matches a domicile
    ColonnesJoueurs_home = ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
           "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10","home_player_11"]


    #Pour recuperer les joueurs qui font partie des 11 titulaires des matches a l'exterieur
    ColonnesJoueurs_away=["away_player_1", "away_player_2", "away_player_3", "away_player_4","away_player_5", "away_player_6", "away_player_7",
                          "away_player_8", "away_player_9","away_player_10", "away_player_11"]


    #avoir les matches joué a domicile pendant la saison en question
    gb=match.groupby("home_team_api_id")
    myteam_home=gb.get_group(idequipe)
    myteam_home=myteam_home[myteam_home.season == saison]


    #avoir les matches joué a l'exterieur pendant la saison en question
    gb2=match.groupby("away_team_api_id")
    myteam_away=gb.get_group(idequipe)
    myteam_away[myteam_away.season == saison]
    playerattributes['date']  = pd.to_datetime(playerattributes['date'])
    df_filtre = playerattributes[(playerattributes.date > datetime.date(int(saison[0:4]),9,1)) & (playerattributes.date < datetime.date(int(saison[5:]),8,30))]
    df_filtre=df_filtre[['player_api_id','overall_rating']]

    
    
    NotesJoueursD=pd.DataFrame()
    for postejoueur in ColonnesJoueurs_home:
        # avoir la liste des joueurs ayant joué dans ce poste

        effectifs=myteam_home[postejoueur].dropna() # on utilise dropna pour eliminer tous les nan
        #print(effectifs)
        NotesJoueursD[postejoueur]=[0]
        noteglob=pd.DataFrame()
        note=list()
        for i in range(len(effectifs)):
            noteglob=pd.DataFrame()

            df_filtre=df_filtre[['player_api_id','overall_rating']]
            note.append(df_filtre[df_filtre['player_api_id']==effectifs.iloc[i]]['overall_rating'].mean())              
            #print(df_masked[df_masked['player_api_id']==effectifs.iloc[i]])
            #NotesJoueurs[postejoueur]=df_masked[df_masked['player_api_id']==effectifs.iloc[i]]['overall_rating']

        noteglob = pd.DataFrame(np.array(note), columns = ["a"]).dropna()

        NotesJoueursD[postejoueur]=[noteglob.mean()[0]]
    #print(NotesJoueursD)

    NotesJoueursA=pd.DataFrame()
    for postejoueur in ColonnesJoueurs_away:
        # avoir la liste des joueurs ayant joué dans ce poste

        effectifs=myteam_away[postejoueur].dropna() # on utilise dropna pour eliminer tous les nan

        NotesJoueursA[postejoueur]=[0]
        noteglob=pd.DataFrame()
        note=list()
        for i in range(len(effectifs)):
            noteglob=pd.DataFrame()

            note.append(df_filtre[df_filtre['player_api_id']==effectifs.iloc[i]]['overall_rating'].mean())              
            #print(df_masked[df_masked['player_api_id']==effectifs.iloc[i]])
            #NotesJoueurs[postejoueur]=df_masked[df_masked['player_api_id']==effectifs.iloc[i]]['overall_rating']
        noteglob = pd.DataFrame(np.array(note), columns = ["a"]).dropna()

        NotesJoueursA[postejoueur]=[noteglob.mean()[0]]
    #print(NotesJoueursA.mean(axis = 1))
    return ((NotesJoueursA.mean(axis = 1)+NotesJoueursD.mean(axis = 1))/2)[0]


la league et les saisons de test

In [23]:
saison_av = "2009/2010" # la saison pour l'apprentissage
saison_ap = "2010/2011" # la saison pour le test
league=1

creation des labels 

In [24]:
lab=matchsaison(saison_av,league)
labT=matchsaison(saison_ap,league)

creation du dataframe contenant les notes des equipes de la league pour la saison et les notes

In [25]:
stats_equipes_league_saison=StatEquipesLeagueSaison(saison_av,1)

C:\Users\Stone\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


In [26]:
stats_equipes_league_saison

,team_api_id,overall_rating
0,65.065166,8203.0
1,65.125857,9999.0
2,65.267966,9984.0
3,65.297502,9986.0
4,65.746646,9997.0
5,65.884718,8571.0
6,65.996210,9987.0
7,65.998372,10001.0
8,66.558906,9993.0
9,66.635297,9994.0


Creaton du labledSet d'apprentissage contenant les notes des equipes, idem pour les données de test

In [47]:
app=LabeledSet(2)
test=LabeledSet(2)
for i in range(lab.size()):  
    
    if(len(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==lab.getX(i)[0]]["overall_rating"])>0):
        NoteD=float(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==lab.getX(i)[0]]["overall_rating"])
    else:
        NoteD=noteEquipe(lab.getX(i)[0],saison_ap)
    
    if(len(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==lab.getX(i)[1]]["overall_rating"])>0):
        NoteE=float(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==lab.getX(i)[1]]["overall_rating"])
    else:        
        NoteE=noteEquipe(lab.getX(i)[1],saison_ap)    
    
    
    if(len(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==labT.getX(i)[0]]["overall_rating"])>0):
        NoteDT=float(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==labT.getX(i)[0]]["overall_rating"])
    else:
        NoteDT=noteEquipe(labT.getX(i)[0],saison_ap)
    
    if(len(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==labT.getX(i)[1]]["overall_rating"])>0):
        NoteET=float(stats_equipes_league_saison[stats_equipes_league_saison["team_api_id"]==labT.getX(i)[1]]["overall_rating"])
    
    else:        
        NoteET=noteEquipe(labT.getX(i)[1],saison_ap)  
    
    
    #print(NoteD,NoteE)
    #print(NoteDT,NoteET)
    app.addExample([NoteD,NoteE],lab.getY(i))
    test.addExample([NoteDT,NoteET],labT.getY(i))  

C:\Users\Stone\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.


Validation du Modéle

In [49]:
print(test.getX(0),test.getY(0))

[[65.41067122 66.20906338]
 [65.86860886 67.82959395]
 [66.46310073 66.11891789]
 [65.57592937 67.65028496]
 [69.02887757 64.95696803]
 [68.37651693 66.49813116]
 [66.02451882 64.66185786]
 [67.64175656 65.82117228]
 [67.82959395 67.65028496]
 [68.37651693 69.02887757]
 [65.41067122 65.57592937]
 [66.49813116 64.66185786]
 [66.11891789 65.82117228]
 [65.86860886 64.95696803]
 [66.46310073 67.64175656]
 [66.20906338 66.02451882]
 [67.65028496 65.86860886]
 [67.64175656 68.37651693]
 [65.82117228 66.20906338]
 [66.49813116 67.82959395]
 [64.66185786 66.11891789]
 [64.95696803 65.41067122]
 [65.57592937 66.46310073]
 [66.02451882 69.02887757]
 [69.02887757 65.57592937]
 [68.37651693 67.65028496]
 [65.41067122 64.66185786]
 [66.11891789 67.82959395]
 [65.86860886 67.64175656]
 [66.46310073 65.82117228]
 [66.20906338 66.49813116]
 [66.02451882 64.95696803]
 [67.82959395 68.37651693]
 [67.65028496 66.46310073]
 [67.64175656 69.02887757]
 [65.82117228 65.41067122]
 [66.49813116 66.11891789]
 

## Partie 5 Résultats

In [17]:
import matplotlib.pyplot as plt
%matplotlib inline 

def plot2DSet(set):
    """ LabeledSet -> NoneType
        HypothÃ¨se: set est de dimension 2
        affiche une reprÃ©sentation graphique du LabeledSet
        remarque: l'ordre des labels dans set peut Ãªtre quelconque
    """
    S_pos = set.x[np.where(set.y == 1),:][0]      # tous les exemples de label +1
    S_neg = set.x[np.where(set.y == -1),:][0]     # tous les exemples de label -1
    S_nul = set.x[np.where(set.y == 0),:][0]     # tous les exemples de label 0
    plt.scatter(S_pos[:,0],S_pos[:,1],marker='o')
    plt.scatter(S_neg[:,0],S_neg[:,1],marker='x')
    plt.scatter(S_nul[:,0],S_nul[:,1],marker='v')

In [18]:
app.size()

210

In [19]:
#plot2DSet(app)
#print("la performance de l'arbre de decision en terme de pourcentage de reussite est de:  "+str(ad.validation(test)*100)+"%")

In [20]:
ad=ArbreDecision(0.9)
ad.train(app) 
#plot_frontiere(app,ad,200)
plot2DSet(app)
print("la performance de l'arbre de decision en terme de pourcentage de reussite est de:  "+str(ad.validation(test)*100)+"%")

NameError: name 'ArbreDecision' is not defined